<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation,InputLayer
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
tf.test.gpu_device_name()

1.15.2


''

In [4]:
#Upload the deu.txt file and put it in the Data Folder
#!ls Data/

deu.txt


In [5]:
#reading lines from thge txt file
# with open('Data/deu.txt','r',encoding='utf-8') as f:
#   lines=(f.readlines())

In [6]:
#Hyperparameters go here
test_size=0.3
m1_lr=0.005
m2_lr=0.005
batch_size=75
m1_lstm_units=64
m2_lstm_units=64
m1_epochs=12
m2_epochs=12
m1_batch_size=128
m2_batch_size=128

In [7]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines,translate_mode):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns dataframe of processed
  English and German sentence pairs
  '''
  proc_lines=[]
  if translate_mode==False:
    for line in lines:
      line=line.strip() #to remove newlines at the end of a sentence
      line=line.split("\t") #splitting by tabs
      line=line[:-1]  #remove contributing information
      
      line[0]=''.join(c for c in unicodedata.normalize('NFD', line[0]) if unicodedata.category(c) != 'Mn')
      line[0]=line[0].encode('utf8','ignore').decode('utf8')
      line[0]=line[0].replace(u'\u200b',' ')
      line[0]=line[0].lower()
      line[0]=line[0].replace('\xa0', ' ')
      line[0]=re.sub(r"([.,?!;])",r" \1 ",line[0])   #adding spaces before and after punctuation
      line[0]=re.sub(r"[0-9]"," ",line[0])
      line[0]=re.sub(r'["]'," ",line[0])
      line[0]=re.sub(r"[']","",line[0])
      line[0]=re.sub(r"[%-]"," ",line[0])
      line[0]=re.sub(r"[:]"," ",line[0])
      line[0]=re.sub(r'[" "]+'," ",line[0])  #removing excess spaces
      line[0]=line[0].strip() #removing spaces from the end of string
      line[0]="<SOS> "+line[0]+" <EOS>"

      line[1]=''.join(c for c in unicodedata.normalize('NFD', line[1]) if unicodedata.category(c) != 'Mn')
      line[1]=line[1].replace(u'\u200b',' ')
      line[1]=line[1].replace('\xa0', ' ')
      line[1]=line[1].lower()
      line[1]=re.sub(r"([.,?!;])",r" \1 ",line[1])
      line[1]=re.sub(r"[0-9]",r" ",line[1])
      line[1]=re.sub(r'["]',"",line[1])
      line[1]=re.sub(r'[—]',"",line[1])
      line[1]=re.sub(r'[„]',"",line[1])
      line[1]=re.sub(r'[“]',"",line[1])
      line[1]=re.sub(r'[–]',"",line[1])
      line[1]=re.sub(r'[‘‚]',"",line[1])
      line[1]=re.sub(r"[']","",line[1])
      line[1]=re.sub(r"[%()-]"," ",line[1])
      line[1]=re.sub(r"[:]"," ",line[1])
      line[1]=re.sub(r'[" "]+'," ",line[1])
      line[1]=line[1].strip()
      line[1]="<SOS> "+line[1]+" <EOS>"

      proc_lines.append(line) 
    return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger'])
  else:
    lines=lines.strip()
    lines=''.join(c for c in unicodedata.normalize('NFD', lines) if unicodedata.category(c) != 'Mn')
    lines=lines.encode('utf8','ignore').decode('utf8')
    lines=lines.replace(u'\u200b',' ')
    lines=lines.lower()
    lines=lines.replace('\xa0', ' ')
    lines=re.sub(r"([.,?!;])",r" \1 ",lines)   #adding spaces before and after punctuation
    lines=re.sub(r"[0-9]"," ",lines)
    lines=re.sub(r'["]'," ",lines)
    lines=re.sub(r"[']","",lines)
    lines=re.sub(r"[%-]"," ",lines)
    lines=re.sub(r"[:]"," ",lines)
    lines=re.sub(r'[" "]+'," ",lines)  #removing excess spaces
    lines=lines.strip() #removing spaces from the end of string
    lines="<SOS> "+lines+" <EOS>"
    return lines

  
     
  

In [8]:
#df=process_text(lines,False)
df=pd.read_csv('data/data.csv')
print(df.head(10))
df[df['eng']=="<SOS> tom will be here soon . how soon ? <EOS>"].iloc[0,1]


                  eng                         ger
0    <SOS> go . <EOS>           <SOS> geh . <EOS>
1    <SOS> hi . <EOS>         <SOS> hallo ! <EOS>
2    <SOS> hi . <EOS>     <SOS> gruß gott ! <EOS>
3   <SOS> run ! <EOS>          <SOS> lauf ! <EOS>
4   <SOS> run . <EOS>          <SOS> lauf ! <EOS>
5   <SOS> wow ! <EOS>    <SOS> potzdonner ! <EOS>
6   <SOS> wow ! <EOS>  <SOS> donnerwetter ! <EOS>
7  <SOS> fire ! <EOS>         <SOS> feuer ! <EOS>
8  <SOS> help ! <EOS>         <SOS> hilfe ! <EOS>
9  <SOS> help ! <EOS>       <SOS> zu hulf ! <EOS>


'<SOS> tom wird bald hier sein . was heißt bald ? <EOS>'

In [35]:
df.tail(10)

,eng,ger
224410,"<SOS> as a prank , some students let three goa...",<SOS> als streich ließen einige schuler drei z...
224411,<SOS> the small crowd at hiroshima peace memor...,"<SOS> um . uhr vormittags , dem genauen zeitpu..."
224412,"<SOS> in todays world , we have to equip all o...",<SOS> in der heutigen welt mussen wir all unse...
224413,<SOS> death is something that were often disco...,"<SOS> wir werden oft davon abgehalten , uber d..."
224414,<SOS> at a moment when our economy is growing ...,"<SOS> in einem moment , in dem unsere wirtscha..."
224415,<SOS> even if some sentences by non native spe...,<SOS> auch wenn satze von nichtmuttersprachler...
224416,<SOS> if someone who doesnt know your backgrou...,"<SOS> wenn jemand , der deine herkunft nicht k..."
224417,<SOS> if someone who doesnt know your backgrou...,"<SOS> wenn jemand fremdes dir sagt , dass du d..."
224418,<SOS> if someone who doesnt know your backgrou...,"<SOS> wenn jemand , der nicht weiß , woher man..."
224419,<SOS> doubtless there exists in this world pre...,<SOS> ohne zweifel findet sich auf dieser welt...


In [9]:
class Vocab_builder():
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    # self.uniq_words.add('<PAD>')
    # self.uniq_words.add('<SOS>')
    # self.uniq_words.add('<EOS>')
    self.maxlen=0
    count=3
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words and word!='<EOS>' and word!='<SOS>'):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=['<PAD>','<SOS>','<EOS>']+sorted(list(self.uniq_words))
    self.num_list=[0,1,2]+self.num_list
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [10]:
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [11]:
eng_w2i,eng_i2w=eng.build_vocab()


Built vocabulary having 15957 elements
Largest sentence length (with tags):113


In [12]:
#checking for special characters in English dictionary
for i in sorted (eng_w2i.keys()) :  
     print(i, end = " ") 

! $ , . / ; <EOS> <PAD> <SOS> ? a aah aardvark aardvarks aarhus abacus abandon abandoned abated abating abbreviation abdicate abdominal abducted abetting abhor abide abiding abilities ability able abnormal abnormality aboard abode abolish abolished abolishing abominably abortion about above abraham abreast abridged abroad abrupt absconded absence absent absolute absolutely absolved absorb absorbed absorbs abstain abstained abstract absurd abundant abuse abused abuses abusing abusive academic accelerated accent accept acceptable accepted accepting accepts access accessible accessories accessory accident accidental accidentally accidents accommodate accommodated accompanied accompanies accompaniment accompany accomplice accomplices accomplish accomplished accomplishes accomplishment according accordion account accountable accountant accounted accounting accounts accumulated accuracy accurate accurately accusation accusations accuse accused accuses accusing accustom accustomed ace aced ac

In [13]:
ger_w2i,ger_i2w=ger.build_vocab()

Built vocabulary having 34809 elements
Largest sentence length (with tags):86


In [14]:
#checking for special characters in German dictionary
for i in sorted (ger_w2i.keys()) :  
     print(i, end = " ") 

! $ + , . / ; <EOS> <PAD> <SOS> ? a aal aale aalglatter aalte aalten aarhus ab abakus abandern abarbeiten abbaubar abbauen abbekommen abbezahlen abbezahlst abbezahlt abbiegen abbiegst abbiegt abbildung abbildungen abbitte abblasen abblitzen abbrechen abbringen abdanken abdankung abdrehen abdruck abdrucke abend abendausgabe abendbrot abende abenden abendessen abendessens abendgarderobe abendkleid abendkleider abendluft abendnachrichten abends abendschule abendvorstellung abendzeitung abenteuer abenteuergeschichte abenteuerlust abenteuerlustig abenteuern aber aberglaube aberglauben aberglaubisch aberkannt abermals abertausende abfahrbereit abfahre abfahren abfahrt abfahrtszeiten abfall abfalle abfallig abfallige abfallproblem abfertigung abfertigungsschalter abfeuerte abfinden abflauens abfliegen abfliegt abflug abfluss abfolge abfuhr abfuhren abgabefrist abgabetermin abgase abgeandert abgebaut abgeben abgeblasen abgebogen abgebrannt abgebrochen abgebruht abgebt abgedeckt abgedreht abged

In [15]:
def sent_to_ind(sentence,lang):
  ind_list=[]
  if lang=='eng':
    tokens=eng.tokenize(sentence)
    for token in tokens:
      ind_list.append(eng_w2i[token])
    while len(ind_list)<eng.maxlen:
      ind_list.append(0)
  else:
    tokens=ger.tokenize(sentence)
    for token in tokens:
      ind_list.append(ger_w2i[token])
    while len(ind_list)<eng.maxlen:
      ind_list.append(0)
    
  return np.array(ind_list)



In [16]:
test_lis=sent_to_ind('<SOS> tom wird bald hier sein . was heißt bald ? <EOS>','ger')
print(len(test_lis))
print(test_lis)

113
[    1 28295 33314  3110 14408 25586     7 32497 13967  3110    10     2
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [17]:
test_lis=sent_to_ind("<SOS> tom will be here soon . how soon ? <EOS>",'eng')
print(len(test_lis))
print(test_lis)

113
[    1 14421 15646  1208  6571 13049     6  6822 13049     9     2     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [18]:
#Shuffling the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [19]:
#splitting the data into training and testing sets

# train_x,test_x,train_y,test_y=train_test_split(df['eng'],df['ger'],test_size=0.1,random_state=42)
# train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.23,random_state=42)


In [23]:
def sent_to_np(series,lang,translate_mode):
  ret_list=[]
  if translate_mode==False :
    if lang=='eng':
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'eng'))
    else:
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'ger'))
    
    ret_list=np.array(ret_list)
    return np.expand_dims(ret_list,axis=2)
  else:
    ans=sent_to_ind(series,'eng')
    ans=np.expand_dims(ans,axis=0)
    ans=np.expand_dims(ans,axis=2)
    return ans


In [24]:
# train_x=sent_to_np(train_x,'eng',False)
# train_y=sent_to_np(train_y,'ger',False)
# test_x=sent_to_np(test_x,'eng',False)
# test_y=sent_to_np(test_y,'ger',False)
# val_x=sent_to_np(val_x,'eng',False)
# val_y=sent_to_np(val_y,'ger',False)


In [25]:
#loading training,testing and validation data
train_x=np.load("data/training/train_x.npy")
train_y=np.load("data/training/train_y.npy")

test_x=np.load("data/testing/test_x.npy")
test_y=np.load("data/testing/test_y.npy")

val_x=np.load("data/validation/val_x.npy")
val_y=np.load("data/validation/val_y.npy")



In [26]:
#Input shape denotes [batch,no.of timesteps,features]
print(train_x.shape)
print(test_x.shape)
print(val_x.shape)

(155523, 113, 1)

In [27]:
def translate(sentence,mod,pad_tol):
  ans=""
  preproc_sent=process_text(sentence,True)
  model_inp=sent_to_np(preproc_sent,'eng',True)
  pred=mod.predict(model_input)
  count=0
  for i in pred[0]:
    ind=np.argmax(i)
    if (ind==0):
      count+=1
    if count<pad_tol:
      ans+=ger_i2w[ind]
    else:
      return ans




In [ ]:
#training starts here ----------------------------------------------------------

In [28]:
def base_LSTM_model(eng_arr,op_vocab_size,lr):
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')
  dense=Dense(op_vocab_size,activation='softmax')
  layer_at_t=TimeDistributed(dense)
  model=tf.keras.Sequential()
  model.add(InputLayer(eng_arr.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(lr),metrics=['accuracy'])
  return model



  

In [54]:
#base_model=base_LSTM_model(train_x,len(ger.vocab_list),m1_lr)

In [55]:
#base_model.fit(train_x,train_y,m1_batch_size,m1_epochs,validation_data=(val_x,val_y))

Train on 155523 samples, validate on 46455 samples
Epoch 1/12
155523/155523 [==============================] - 658s 4ms/sample - loss: 0.5786 - acc: 0.9275 - val_loss: 0.4472 - val_acc: 0.9302
Epoch 2/12
155523/155523 [==============================] - 655s 4ms/sample - loss: 0.4451 - acc: 0.9302 - val_loss: 0.4423 - val_acc: 0.9313
Epoch 3/12
155523/155523 [==============================] - 661s 4ms/sample - loss: 0.4406 - acc: 0.9306 - val_loss: 0.4399 - val_acc: 0.9312
Epoch 4/12
155523/155523 [==============================] - 658s 4ms/sample - loss: 0.4375 - acc: 0.9309 - val_loss: 0.4374 - val_acc: 0.9316
Epoch 5/12
155523/155523 [==============================] - 659s 4ms/sample - loss: 0.4362 - acc: 0.9310 - val_loss: 0.4370 - val_acc: 0.9310
Epoch 6/12
155523/155523 [==============================] - 661s 4ms/sample - loss: 0.4344 - acc: 0.9311 - val_loss: 0.4350 - val_acc: 0.9313
Epoch 7/12
155523/155523 [==============================] - 657s 4ms/sample - loss: 0.4330 - acc:

In [58]:
base_model=tf.keras.models.load_model("model/MT_base_RNN.h5")
base_model.summary()

In [60]:
l,acc=base_model.evaluate(test_x,test_y)
print("Base model loss for testing set:{}".format(l))
print("Base model accuracy for testing set:{}".format(acc))

22442/22442 [==============================] - 101s 5ms/sample - loss: 0.4346 - acc: 0.9315
Base model loss:0.43460041285623535
Base model accuracy:0.9315182566642761


In [76]:
translate("hello there how are you doing",base_model)

In [79]:
def embedding_LSTM(eng_arr,op_vocab_size,lr):
  

(1, 113, 34809)


array([[[4.3552777e-16, 9.9999917e-01, 4.6834449e-19, ...,
         9.8062751e-16, 6.6480778e-16, 9.3407278e-18],
        [1.2074879e-05, 1.5622851e-07, 1.7145994e-07, ...,
         1.5769974e-08, 6.8593167e-08, 3.8900536e-09],
        [2.3691875e-03, 2.9970595e-10, 2.1579079e-03, ...,
         6.5130521e-08, 7.6315371e-07, 2.1661398e-08],
        ...,
        [9.9999309e-01, 3.6652548e-22, 6.2503756e-07, ...,
         2.3464938e-13, 1.8469766e-13, 2.0448496e-13],
        [9.9999309e-01, 3.6652548e-22, 6.2503756e-07, ...,
         2.3464938e-13, 1.8469766e-13, 2.0448496e-13],
        [9.9999309e-01, 3.6652548e-22, 6.2503756e-07, ...,
         2.3464938e-13, 1.8469766e-13, 2.0448496e-13]]], dtype=float32)